In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfail'], pff=False)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350, resolution=N

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable_name=['3_RfR 0'], pff=False)
IntegerParameter('4_RfR 0', 0, 1, resolution=None, default=None, variable_name=['4_RfR 0'], pff=False)
IntegerParameter('EWS_DaysToThreat', 0, 4, resolution=None, default=None, variable_name=['EWS_DaysToThreat'], pff=False)
IntegerParameter('A.1_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.1_DikeIncrease 0'], pff=False)
IntegerParameter('A.2_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.2_DikeIncrease 0'], pff=False)
IntegerParameter('A.3_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.3_DikeIncrease 0'

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1_Expected Annual Damage', variable_name=('A.1_Expected Annual Damage',), function=<function sum_over at 0x00000268741D79D0>)
ScalarOutcome('A.1_Dike Investment Costs', variable_name=('A.1_Dike Investment Costs',), function=<function sum_over at 0x00000268741D79D0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x00000268741D79D0>)
ScalarOutcome('A.2_Expected Annual Damage', variable_name=('A.2_Expected Annual Damage',), function=<function sum_over at 0x00000268741D79D0>)
ScalarOutcome('A.2_Dike Investment Costs', variable_name=('A.2_Dike Investment Costs',), function=<function sum_over at 0x00000268741D79D0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x00000268741D79D0>)
ScalarOutcome('A.3_Expected Annual Damage', variable_name=('A.3_Expected Annual Damage',), function=<function sum_over at 0x0000026874

In [8]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:07<00:00, 25.57it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage', 'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths', 'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs', 'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage', 'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths', 'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,4_RfR 0,A.1_DikeIncrease 0,A.2_DikeIncrease 0,A.3_DikeIncrease 0,A.4_DikeIncrease 0,A.5_DikeIncrease 0,EWS_DaysToThreat,scenario,policy,model
0,96,201.007723,1.0,0.359837,220.394474,10.0,0.278565,118.905859,1.5,0.043075,...,0,5,7,5,0,2,4,4,0,dikesnet
1,14,267.827423,1.5,0.412654,84.571942,1.5,0.976825,57.293274,10.0,0.968151,...,0,5,7,5,0,2,4,5,0,dikesnet
2,17,131.657248,1.5,0.335902,89.852792,1.0,0.626389,72.533046,1.5,0.981480,...,0,5,7,5,0,2,4,6,0,dikesnet
3,12,252.212238,10.0,0.629750,234.397134,10.0,0.286536,192.672252,1.0,0.565048,...,0,5,7,5,0,2,4,7,0,dikesnet
4,113,37.175089,1.5,0.956747,51.746330,1.5,0.954082,48.363715,10.0,0.591046,...,0,5,7,5,0,2,4,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,21,72.127274,10.0,0.523714,56.722063,1.5,0.796150,254.312751,1.0,0.411767,...,1,2,2,10,7,0,1,49,3,dikesnet
196,73,302.157419,1.5,0.765641,44.760587,10.0,0.857290,164.680743,10.0,0.648218,...,1,2,2,10,7,0,1,50,3,dikesnet
197,9,276.931466,10.0,0.645717,192.436076,1.0,0.727163,81.855123,1.5,0.151935,...,1,2,2,10,7,0,1,51,3,dikesnet
198,43,158.487451,10.0,0.393216,104.727593,1.0,0.554560,325.769167,1.5,0.480425,...,1,2,2,10,7,0,1,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Dike Investment Costs,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Dike Investment Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0.0,5.397251e+07,0.0,0.000000e+00,7.331545e+07,0.000000,2.897660e+07,2.879840e+07,0.004944,2.529971e+06,0,0.000130,0.000000e+00,2.757823e+07,0.000000,30700000.0,1330.728755
1,0.0,5.397251e+07,0.0,0.000000e+00,7.331545e+07,0.000000,0.000000e+00,2.879840e+07,0.000000,3.397446e+06,0,0.000319,0.000000e+00,2.757823e+07,0.000000,30700000.0,364.084667
2,0.0,5.397251e+07,0.0,0.000000e+00,7.331545e+07,0.000000,0.000000e+00,2.879840e+07,0.000000,1.325076e+06,0,0.000101,4.570347e+07,2.757823e+07,0.005504,30700000.0,3787.561353
3,0.0,5.397251e+07,0.0,0.000000e+00,7.331545e+07,0.000000,0.000000e+00,2.879840e+07,0.000000,3.045746e+06,0,0.000278,0.000000e+00,2.757823e+07,0.000000,30700000.0,334.571764
4,0.0,5.397251e+07,0.0,0.000000e+00,7.331545e+07,0.000000,0.000000e+00,2.879840e+07,0.000000,1.112436e+07,0,0.000906,1.722391e+07,2.757823e+07,0.002593,30700000.0,3159.189712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,3.725520e+07,0.0,0.000000e+00,4.281842e+07,0.000000,0.000000e+00,4.421502e+07,0.000000,0.000000e+00,14261651,0.000000,1.147255e+07,0.000000e+00,0.005321,679700000.0,493.631543
196,0.0,3.725520e+07,0.0,0.000000e+00,4.281842e+07,0.000000,0.000000e+00,4.421502e+07,0.000000,0.000000e+00,14261651,0.000000,1.489784e+06,0.000000e+00,0.000546,679700000.0,49.273298
197,0.0,3.725520e+07,0.0,0.000000e+00,4.281842e+07,0.000000,0.000000e+00,4.421502e+07,0.000000,0.000000e+00,14261651,0.000000,8.835286e+06,0.000000e+00,0.003199,679700000.0,295.250256
198,0.0,3.725520e+07,0.0,4.222899e+06,4.281842e+07,0.001637,0.000000e+00,4.421502e+07,0.000000,0.000000e+00,14261651,0.000000,2.166216e+07,0.000000e+00,0.007797,679700000.0,848.051501


In [11]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "Baseline",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
    Policy(
        "DI max",
        **dict(
            get_do_nothing_dict(),
            **{"A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5}
        )
    ),
        Policy(
        "RfR max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "EWS max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0, "2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':4
               }
        )
    ),
        Policy(
        "All max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5,
               'EWS_DaysToThreat':4
               }
        )
    ),
        Policy(
        "Di-Ge RfR-Ov max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "DiGe RfROv mod",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb Green",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb Grey",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 4,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb GreenEWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        )
    )           
]

policy_old = [Policy(
        "Policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    )]


In [12]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 10 policies * 1 model(s) = 1000 experiments
100%|██████████████████████████████████████| 1000/1000 [00:35<00:00, 28.34it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [13]:
experiments, outcomes = results
print(experiments.keys())
experiments.head()

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', '0_RfR 0', '1_RfR 0',
       '2_RfR 0', '3_RfR 0', '4_RfR 0', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.2_DikeIncrease 0', 'A.3_DikeIncrease 0',
       'A.4_DikeIncrease 0', 'A.5_DikeIncrease 0', 'scenario', 'policy',
       'model'],
      dtype='object')


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,4_RfR 0,EWS_DaysToThreat,A.1_DikeIncrease 0,A.2_DikeIncrease 0,A.3_DikeIncrease 0,A.4_DikeIncrease 0,A.5_DikeIncrease 0,scenario,policy,model
0,33,178.554495,1.0,0.709712,115.707874,10.0,0.543617,66.459464,1.0,0.880262,...,0,0,0,0,0,0,0,54,Baseline,dikesnet
1,109,336.234365,10.0,0.035067,322.731344,1.0,0.266965,310.232145,10.0,0.058370,...,0,0,0,0,0,0,0,55,Baseline,dikesnet
2,92,235.701073,1.0,0.571703,206.161233,10.0,0.597315,294.903324,10.0,0.243594,...,0,0,0,0,0,0,0,56,Baseline,dikesnet
3,80,112.260608,10.0,0.460747,117.467073,10.0,0.349074,242.605875,10.0,0.995038,...,0,0,0,0,0,0,0,57,Baseline,dikesnet
4,19,168.271769,1.5,0.584757,239.368355,10.0,0.158113,81.273040,10.0,0.222703,...,0,0,0,0,0,0,0,58,Baseline,dikesnet


In [14]:
from funs_viz import tidy_results

results_df = tidy_results(results,6,1)
results_df.to_csv("results/test_1.csv")

print(results_df.columns)
results_df.head()

Index(['A.0_ID flood wave shape', 'discount rate 0', 'EWS_DaysToThreat',
       'scenario', 'policy', 'model', 'RfR Total Costs',
       'Expected Evacuation Costs', 'Dike ring', 'Bmax', 'Brate', 'pfail',
       'RfR 0', 'DikeIncrease 0', 'Dike Investment Costs',
       'Expected Annual Damage', 'Expected Number of Deaths'],
      dtype='object')


,A.0_ID flood wave shape,discount rate 0,EWS_DaysToThreat,scenario,policy,model,RfR Total Costs,Expected Evacuation Costs,Dike ring,Bmax,Brate,pfail,RfR 0,DikeIncrease 0,Dike Investment Costs,Expected Annual Damage,Expected Number of Deaths
0,33,3.5,0,54,Baseline,dikesnet,0.0,0.0,A.1,178.554495,1.0,0.709712,0,0,0,1.527104e+07,0.012081
1,109,4.5,0,55,Baseline,dikesnet,0.0,0.0,A.1,336.234365,10.0,0.035067,0,0,0,4.899202e+08,0.455529
2,92,1.5,0,56,Baseline,dikesnet,0.0,0.0,A.1,235.701073,1.0,0.571703,0,0,0,7.933249e+07,0.027691
3,80,3.5,0,57,Baseline,dikesnet,0.0,0.0,A.1,112.260608,10.0,0.460747,0,0,0,1.100535e+08,0.083233
4,19,4.5,0,58,Baseline,dikesnet,0.0,0.0,A.1,168.271769,1.5,0.584757,0,0,0,2.866670e+07,0.028649
